# Day 23
## Part 1

In [10]:
from collections import defaultdict, namedtuple
import pyrsistent as pyr

Edge = namedtuple('Edge', 'neighbour, length')

def create_graph():
    g = defaultdict(list)
    top_row = [1,2,4,6,8,10,11]
    for i in range(len(top_row) - 1):
        g[str(top_row[i])].append(Edge(str(top_row[i + 1]), top_row[i + 1] - top_row[i]))
        g[str(top_row[i + 1])].append(Edge(str(top_row[i]), top_row[i + 1] - top_row[i]))
    for xs, y in [
        (("2", "4"), "A1"),
        (("4", "6"), "B1"),
        (("6", "8"), "C1"),
        (("8", "10"), "D1")
    ]:
        for x in xs:
            g[x].append(Edge(y, 2))
            g[y].append(Edge(x, 2))
    for c in "ABCD":
        g[f"{c}1"].append(Edge(f"{c}2", 1))
        g[f"{c}2"].append(Edge(f"{c}1", 1))
    return g

def parse_data(s):
    lines = s.strip().splitlines()
    room_locations = {
        "A1": (2, 3),
        "A2": (3, 3),
        "B1": (2, 5),
        "B2": (3, 5),
        "C1": (2, 7),
        "C2": (3, 7),
        "D1": (2, 9),
        "D2": (3, 9)
    }
    burrow = pyr.pmap()
    for room in room_locations:
        row, col = room_locations[room]
        burrow = burrow.set(room, lines[row][col])
    return burrow

test_data = parse_data('''
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
''')

test_data

pmap({'D1': 'D', 'A2': 'A', 'C2': 'C', 'D2': 'A', 'C1': 'B', 'B1': 'C', 'B2': 'D', 'A1': 'B'})